<a href="https://colab.research.google.com/github/SushmitalKhan/Dissertation/blob/main/powerSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive
import os
from glob import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# `READ ALL FILES`



In [ ]:
# Define the folder containing CSV files
folder_path = "/content/drive/MyDrive/Dissertation/Study 1/googleData/CSV_data"  # Change this to your folder path

# Get a list of all CSV files in the folder
csv_files = glob(os.path.join(folder_path, "*.csv"))  # Find all CSV files

df_names = []  # Store DataFrame names
search_title_dfs = []  # List to store "Search Title" columns

# `MAKING POWERSET WHERE ALL THE SEARCH TITLES FOR ALL ACTIVITY AND YEAR IS IN ONE BIG CSV`

In [ ]:

for file in csv_files:
    output_filename = os.path.splitext(os.path.basename(file))[0]  # Extract filename without extension
    globals()[output_filename] = pd.read_csv(file)  # Create a variable with the filename

    # Get columns that contain "Search Title" and "Search Date"
    search_title_cols = [col for col in globals()[output_filename].columns if "Search Title" in col]
    search_date_cols = [col for col in globals()[output_filename].columns if "Search Date" in col]

    if search_title_cols and search_date_cols:  # Check if there are matching columns
        # Extract relevant columns
        search_title_df = globals()[output_filename][search_title_cols + search_date_cols]

        # Convert "Search Date" columns to datetime and extract year
        for date_col in search_date_cols:
            search_title_df[date_col] = pd.to_datetime(search_title_df[date_col], errors='coerce').dt.year

        # Lowercase and remove extra spaces or characters
        for col in search_title_cols:
            search_title_df[col] = search_title_df[col].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

        # Drop NaN values after the transformations
        search_title_df.dropna(subset=search_title_cols, inplace=True)

        # Drop duplicates
        search_title_df.drop_duplicates(inplace=True)

        # Rename columns by prefixing the filename
        search_title_df.columns = [f"{output_filename}_{col}" for col in search_title_df.columns]

        # Append to the list of dataframes
        search_title_dfs.append(search_title_df)

    df_names.append(output_filename)  # Store DataFrame name

# Combine all extracted "Search Title" and "Search Date" columns into one DataFrame
if search_title_dfs:
    combined_search_titles_df = pd.concat(search_title_dfs, axis=1)

# Print confirmation
print(f"Loaded {len(csv_files)} CSV files. Extracted 'Search Title' and 'Search Date' columns into a new DataFrame.")
print("DataFrames created:", df_names)  # Print names of all DataFrames

# Display the first few rows of the combined DataFrame
# print(combined_search_titles_df.head())  # Optionally display combined DataFrame


In [ ]:
output_filename = "/content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_all_year_col.csv"

combined_search_titles_df.to_csv(output_filename, index=False, encoding='utf-8')

print(f"Data saved to {output_filename}")

Data saved to /content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_all_year_col.csv


In [ ]:

for file in csv_files:
    output_filename = os.path.splitext(os.path.basename(file))[0]  # Extract filename without extension
    globals()[output_filename] = pd.read_csv(file)  # Create a variable with the filename

    # Get columns that contain "Search Title" and "Search Date"
    search_title_cols = [col for col in globals()[output_filename].columns if "Search Title" in col]
    search_date_cols = [col for col in globals()[output_filename].columns if "Search Date" in col]

    if search_title_cols and search_date_cols:  # Check if there are matching columns
        # Extract relevant columns
        search_title_df = globals()[output_filename][search_title_cols + search_date_cols]

        # Convert "Search Date" columns to datetime and extract year
        for date_col in search_date_cols:
            search_title_df[date_col] = pd.to_datetime(search_title_df[date_col], errors='coerce').dt.year

        # Lowercase and remove extra spaces or characters
        for col in search_title_cols:
            search_title_df[col] = search_title_df[col].str.lower().str.replace(r'\s+', ' ', regex=True).str.strip()

        # Drop NaN values after the transformations
        search_title_df.dropna(subset=search_title_cols, inplace=True)

        # Drop duplicates
        search_title_df.drop_duplicates(inplace=True)

        # Rename columns by prefixing the filename
        search_title_df.columns = [f"{output_filename}_{col}" for col in search_title_df.columns]

        # Append to the list of dataframes
        search_title_dfs.append(search_title_df)

    df_names.append(output_filename)  # Store DataFrame name

# Combine all extracted "Search Title" and "Search Date" columns into one DataFrame
if search_title_dfs:
    combined_search_titles_df = pd.concat(search_title_dfs, axis=1)

# Print confirmation
print(f"Loaded {len(csv_files)} CSV files. Extracted 'Search Title' and 'Search Date' columns into a new DataFrame.")
print("DataFrames created:", df_names)  # Print names of all DataFrames

# Display the first few rows of the combined DataFrame
# print(combined_search_titles_df.head())  # Optionally display combined DataFrame


In [ ]:
combined_search_titles_df

# `DROPPING COLUMNS THAT HAS LESS THAN 200 DATA POINTS`

In [ ]:
# Ensure that the 'Search Date' columns (that are now years) exist in the DataFrame
for date_col in combined_search_titles_df.columns:
    if "Search Date" in date_col:
        # Filter data by year (since it's in the 'Search Date' column)
        for year in combined_search_titles_df[date_col].unique():
            # Filter the DataFrame by the current year
            year_filtered_df = combined_search_titles_df[combined_search_titles_df[date_col] == year]

            # Save to a CSV file, naming it based on the year
            # output_filename = f"/content/drive/MyDrive/Dissertation/Study 1/Powerset/search_titles_{year}.csv"
            year_filtered_df.to_csv(output_filename, index=False)
            print(f"Saved data for year {year} to {output_filename}")

In [ ]:
# Get the count of non-NaN values for each column
column_lengths = combined_search_titles_df.count()

# Filter columns that have at least 200 non-NaN values
columns_to_keep = column_lengths[column_lengths >= 200].index

# Drop columns that don't meet the criteria
combined_search_titles_df = combined_search_titles_df[columns_to_keep]

# Print the updated DataFrame
print(combined_search_titles_df.head())

In [ ]:
output_filename = "/content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_col200++.csv"

combined_search_titles_df.to_csv(output_filename, index=False, encoding='utf-8')

print(f"Data saved to {output_filename}")

Data saved to /content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_col200++.csv


# `FILTERING SEARCH TITLES FOR GIVEN ACTIVITY BY YEAR, AND SAVING SEARCH TITLES FILTERED BY YEARS AS SEPARATE CSV`

In [ ]:
# Define the folder containing CSV files
# folder_path = "/content/drive/MyDrive/Dissertation/Study 1/googleData/CSV_data"  # Change this to your folder path

# Get a list of all CSV files in the folder
# csv_files = glob(os.path.join(folder_path, "*.csv"))  # Find all CSV files

# df_names = []

# Read each CSV file and assign it to a variable with the same name as the file
for file in csv_files:
    output_filename = os.path.splitext(os.path.basename(file))[0]  # Extract filename without extension
    globals()[output_filename] = pd.read_csv(file)  # Create a variable with the filename

    # Drop duplicates from the "Search Title" column
    if "Search Title" in globals()[output_filename].columns:
        globals()[output_filename].drop_duplicates(subset=["Search Title"], inplace=True)
        globals()[output_filename].dropna(subset=["Search Title"], inplace=True)

    # Rename columns by prefixing with filename
    globals()[output_filename].columns = [f"{output_filename}_{col}" for col in globals()[output_filename].columns]

    df_names.append(output_filename)  # Store DataFrame name


# Print confirmation
print(f"Loaded {len(csv_files)} CSV files as separate DataFrames.")
print("DataFrames created:", df_names)  # Print names of all DataFrames

Loaded 10 CSV files as separate DataFrames.
DataFrames created: ['takeout1_videoSearch_MyActivity', 'takeout1_imageSearch_MyActivity', 'takeout1_chrome_MyActivity', 'takeout1_gNews_MyActivity', 'takeout1_books_MyActivity', 'takeout1_playstore_MyActivity', 'takeout1_maps_MyActivity', 'takeout1_YT_search-history', 'takeout1_YT_watch-history', 'takeout1_playGames_MyActivity']


In [ ]:
# Define output folder
output_folder = "/content/drive/MyDrive/Dissertation/Study 1/Filtered by Year"
os.makedirs(output_folder, exist_ok=True)  # Ensure output directory exists

def process_dataframe(df, df_name):
    """
    Processes a given DataFrame:
    - Extracts 'Search Title' and 'Search Date' columns
    - Filters data by year
    - Saves each year's 'Search Title' in separate CSV files

    Parameters:
    - df: DataFrame to process
    - df_name: Name of the original DataFrame (for file naming)
    """
    # Identify relevant columns
    search_title_cols = [col for col in df.columns if "Search Title" in col]
    search_date_cols = [col for col in df.columns if "Search Date" in col or "Date" in col]

    if not search_title_cols or not search_date_cols:
        print(f"Skipping {df_name}: No valid 'Search Title' or 'Search Date' column found.")
        return

    # Extract necessary columns
    search_data = df[search_title_cols + search_date_cols].copy()

    # Convert date column to datetime and extract year
    date_col = search_date_cols[0]  # Assume first matching column is correct
    search_data[date_col] = pd.to_datetime(search_data[date_col], errors='coerce').dt.year

    # Drop NaN values in Search Title
    search_data.dropna(subset=search_title_cols, inplace=True)

    # Remove duplicates
    search_data.drop_duplicates(inplace=True)

    # Get unique years
    unique_years = search_data[date_col].dropna().unique()

    for year in unique_years:
        # Filter data for the specific year
        year_data = search_data[search_data[date_col] == year][search_title_cols]

        # Save to CSV
        output_filename = f"{df_name}_search_titles_{year}.csv"
        output_path = os.path.join(output_folder, output_filename)
        year_data.to_csv(output_path, index=False)

        print(f"Saved: {output_filename} ({len(year_data)} rows)")

# List of DataFrame names
df_names = [
    "takeout1_videoSearch_MyActivity",
    "takeout1_imageSearch_MyActivity",
    "takeout1_chrome_MyActivity",
    "takeout1_gNews_MyActivity",
    "takeout1_books_MyActivity",
    "takeout1_playstore_MyActivity",
    "takeout1_maps_MyActivity",
    "takeout1_YT_search-history",
    "takeout1_YT_watch-history",
    "takeout1_playGames_MyActivity"
]

# Process each DataFrame
for df_name in df_names:
    if df_name in globals():
        process_dataframe(globals()[df_name], df_name)
    else:
        print(f"Skipping {df_name}: DataFrame not found in memory.")


In [ ]:
import pandas as pd
import glob
import os

# Folder path containing the CSV files
folder_path = "/content/drive/MyDrive/Dissertation/Study 1/Filtered by Year/2016"  # Replace with the actual folder path

# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Read and concatenate all CSV files into one DataFrame
df_combined = pd.concat((pd.read_csv(file) for file in csv_files), ignore_index=True)

# Sort each column so that non-NaN values appear first
df_sorted = df_combined.apply(lambda col: col.dropna().tolist() + [None] * col.isna().sum(), axis=0)


# Save the combined DataFrame (optional)
df_sorted.to_csv("/content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_by_year/allActivity_2016.csv", index=False)

# Display the first few rows
# print(df_combined.head())


In [ ]:
df_sorted

In [ ]:
non_nan_counts = df_combined.count()